<a href="https://colab.research.google.com/github/Ektaroy/PythonProjects/blob/main/blurImageEnhancement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

os.environ["KAGGLE_USERNAME"] = "deeptiredhu"
os.environ["KAGGLE_KEY"] = "672964425943bc92bde476bf4eae6ce6"

!kaggle datasets download kwentar/blur-dataset

blur-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display

In [ ]:
!unzip blur-dataset.zip

Archive:  blur-dataset.zip
replace blur_dataset_scaled/defocused_blurred/0_IPHONE-SE_F.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import cv2

In [ ]:
ROOT_DIR = "defocused_blurred"
IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128
TOTAL_IMAGES = 350

In [ ]:
images = os.listdir(ROOT_DIR)

In [ ]:
len(images)

In [ ]:
images_list = []
for i in tqdm(range(TOTAL_IMAGES)):
  img = cv2.imread(ROOT_DIR + "/" + images[i])
  img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
  img = img / 255.0 # normalize image
  images_list.append(img)

images_arr = np.array(images_list)

In [ ]:
images_arr.shape

In [ ]:
plt.figure(1, figsize=(10,10))
for i in range(25):
  plt.subplot(5,5,i+1)
  plt.imshow(images_arr[i])
  plt.axis("off")
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU, Reshape, Dropout, Flatten
from tensorflow.keras.models import Model

In [ ]:
NOISE_DIM = 100
CHANNELS = 3

In [ ]:
def discriminator_model():
  inp = Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
  x = Conv2D(32, (3,3))(inp)
  x = LeakyReLU()(x)

  x = Conv2D(64, (3,3), strides = 2)(x)
  x = LeakyReLU()(x)

  x = Conv2D(128, (3,3), strides = 2)(x)
  x = LeakyReLU()(x)

  x = Conv2D(256, (3,3), strides = 2)(x)
  x = LeakyReLU()(x)

  x = Flatten()(x)
  x = Dropout(0.3)(x)

  oup = Dense(1, activation="sigmoid")(x)

  model = Model(inp, oup)
  return model

In [ ]:
discriminator = discriminator_model()

In [ ]:
discriminator.summary()

In [ ]:
def generator_model():
  gen_input = Input(shape=(NOISE_DIM, ))
  x = Dense(256 * 14* 14)(gen_input)
  x = LeakyReLU()(x)
  x = Reshape((14,14,256))(x)

  x = Conv2D(256, (3,3), padding="same")(x)
  x = LeakyReLU()(x)

  x = Conv2DTranspose(128, 4, strides = 2)(x)
  x = LeakyReLU()(x)

  x = Conv2DTranspose(64, 4, strides = 2)(x)
  x = LeakyReLU()(x)

  x = Conv2DTranspose(32, 4, strides = 2)(x)
  x = LeakyReLU()(x)

  x = Conv2DTranspose(32, 5, strides=1)(x)
  out = Conv2D(CHANNELS, 3)(x)

  model = Model(gen_input, out)
  return model

In [ ]:
generator = generator_model()

In [ ]:
generator.summary()

In [ ]:
noise = tf.random.normal([1,100])

In [ ]:
fake_img = generator(noise)

In [ ]:
plt.imshow(fake_img[0,:,:,0])
plt.show()

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
#BATCH_SIZE = 10

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Produce images for the GIF as you go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
train(dataset, EPOCHS)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)